In [3]:
# We import pytorch and torchvision
import torchvision, torch
import torch.nn as nn
import torch.nn.functional as F
# Matplotlib is a library to plot graphs in python
import matplotlib.pyplot as plt
# Import numpy
import numpy as np

# We load CIFAR-10 dataset
train_dataset = torchvision.datasets.CIFAR10(root='data/', train=True, transform=torchvision.transforms.ToTensor(), download=True)
test_dataset = torchvision.datasets.CIFAR10(root='data/', train=False, transform=torchvision.transforms.ToTensor(), download=True)

# We print the first picture of the dataset with matplotlib
def get_and_print_first_image(dataset):
    # Je recupère avec un iterateur le premier élément du dataset
    image = next(iter(dataset))
    # We print the image with pyplot
    plt.imshow(np.transpose(image[0], (1, 2, 0)))
    return image

# We print the 3 colors channels with matplotlib from Tensor version of the image
def print_3_channels_from_tensor_image(image):
    fig, axs = plt.subplots(1, 3, figsize=(15, 5))
    # We show the 3 channels of the image
    for i in range(3):
        axs[i].imshow(image[0][i])
        axs[i].set_title(train_dataset.classes[image[1]])
        
# We print the first image of the dataset
#image = get_and_print_first_image(train_dataset)
# We print the 3 colors channels of the image
#print_3_channels_from_tensor_image(image)

# Charger un dataloader with batch size x
def get_data_loader(dataset, batch_size):
    return torch.utils.data.DataLoader(dataset, batch_size=batch_size, shuffle=True)

def get_batch_format(data_loader):
    imagesT, labelsT = next(iter(data_loader))
    print('images.shape:', imagesT.shape)
    print('labels.shape:', labelsT.shape)

class MyClass(nn.Module):
    def __init__(self, input_dim, hidden_dim1, hidden_dim2, output_dim):
        super(MyClass, self).__init__()
        self.a = nn.Linear(input_dim, hidden_dim1)
        self.b = nn.Linear(hidden_dim1, hidden_dim2)
        self.c = nn.Linear(hidden_dim2, output_dim)

    def forward(self, x):
        x = self.a(x)
        x = self.b(x)
        x = self.c(x)
        return x

# Fonction accuracy pour calculer le total des bonnes réponses
def accuracy(predictions, labels):
    _, predicted_labels = torch.max(predictions, 1)
    correct = (predicted_labels == labels).sum().item()
    return correct / len(labels)

# Function of test to understand
def test_to_understand(train_loader):
    batch = next(iter(train_loader))
    fc = nn.Linear(3*32*32,10)
    # We launch on batch[0] the fully connected layer
    imagesTrain, labelsTrain = batch
    imagesTrain = imagesTrain.view(imagesTrain.shape[0], -1)
    print(imagesTrain.shape)

    # We launch the fully connected layer
    output = fc(imagesTrain)
    print(output.shape)
    
    # On crée une instance de la classe MyClass
    model = MyClass(3*32*32, 32, 24, 10)
    # On lance le modèle sur les images
    output = model(imagesTrain)
    # On applique la fonction softmax sur les prédictions
    probs = F.softmax(output, dim=1)
    maxProbs, index= torch.max(probs, dim=1)
    print(maxProbs)

    # 1. Récupérez le "label" de la première image
    first_image_label = labelsTrain[0]
    print(f"Label of the first image: {first_image_label}")

    # Calculer l'accuracy pour le batch actuel
    batch_accuracy = accuracy(output, labelsTrain)
    print(f"Batch accuracy: {batch_accuracy * 100:.2f}%")
    lossFn = F.cross_entropy
    # Calculer la loss pour le batch actuel
    loss = lossFn(probs, labelsTrain)
    print(f"Loss: {loss.item()}")

# Charger dans un DataLoader le data set de train CIFAR10
train_loader = get_data_loader(train_dataset, 32) # On charge par batch de 32

# On charge le dataset de test
test_loader = get_data_loader(test_dataset, 10000) # On charge tout le dataset de test

# Print the format of the batch
get_batch_format(train_loader)

# On crée une instance de la classe MyClass
model = MyClass(3*32*32, 32, 24, 10)
# On crée une fonction de loss
lossFn = F.cross_entropy
learningRate = 0.01
# On crée un optimiseur SGD
opt = torch.optim.SGD(model.parameters(), lr=learningRate)

def fit_one_cycle(model, train_loader, test_loader, epoch, num_epochs): 
    model.train()
    for x, y in train_loader:
        preds = model(x.view(x.shape[0], -1))
        loss = lossFn(preds, y)
        # Compute gradients
        loss.backward()
        # Update parameters
        opt.step()
        # Reset gradients to 0
        opt.zero_grad()
    
    # Validation
    model.eval()
    with torch.no_grad():
        imagesTest, labelsTest = next(iter(test_loader))
        imagesTest = imagesTest.view(imagesTest.shape[0], -1)
        val_preds = model(imagesTest)
        val_loss = lossFn(val_preds, labelsTest)
        val_acc = accuracy(val_preds, labelsTest)
        print(f"Epoch {epoch+1}/{num_epochs}, Loss: {val_loss.item()}, Accuracy: {val_acc * 100:.2f}%")

# Training loop
num_epochs = 10
for epoch in range(num_epochs):
    fit_one_cycle(model, train_loader, test_loader, epoch, num_epochs)

Files already downloaded and verified
Files already downloaded and verified
images.shape: torch.Size([32, 3, 32, 32])
labels.shape: torch.Size([32])
Epoch 1/10, Loss: 1.8796218633651733, Accuracy: 34.19%
Epoch 2/10, Loss: 1.834290862083435, Accuracy: 34.72%
Epoch 3/10, Loss: 1.7996501922607422, Accuracy: 35.45%
Epoch 4/10, Loss: 1.8411287069320679, Accuracy: 33.95%
Epoch 5/10, Loss: 1.7602410316467285, Accuracy: 38.46%
Epoch 6/10, Loss: 1.7410898208618164, Accuracy: 39.42%
Epoch 7/10, Loss: 1.7976741790771484, Accuracy: 36.37%
Epoch 8/10, Loss: 1.7650412321090698, Accuracy: 37.52%
Epoch 9/10, Loss: 1.7427005767822266, Accuracy: 39.12%
Epoch 10/10, Loss: 1.7688114643096924, Accuracy: 37.18%
